In [1]:
!pip install torch_geometric_temporal
!pip install -U torch-geometric  # or pip install torch-geometric --upgrade
!sed -i 's/from torch_geometric.utils.to_dense_adj import to_dense_adj/from torch_geometric.utils.dense import to_dense_adj/' /usr/local/lib/python3.11/dist-packages/torch_geometric_temporal/nn/attention/tsagcn.py
!sed -i 's/from torch_geometric.utils.dense import to_dense_adj/from torch_geometric.utils import to_dense_adj/' /usr/local/lib/python3.11/dist-packages/torch_geometric_temporal/nn/attention/tsagcn.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 90.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.4 MB/s eta 0:00:00
  Created wheel for torch_geometric_temporal: filename=torch_geometric_temporal-0.54.0-py3-none-any.whl size=86709 sha256=dfbd76a4ef8117c422223882c2ebcc2d09b73657249ae55579749a8ed3d6b210
  Stored in directory: /root/.cache/pip/wheels/9e/9b/b6/e15256e053f0cb49b1084a67a709db909d418386a231f0722c
  Created wheel for torch_scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.wh

In [2]:
!pip install torch_geometric

In [3]:
!pip install pydgn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 107.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.29.0 requires pandas>=1.5.3, but you have pandas 1.3.5 which is incompatible.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.3.5 which is incompatible.
woodwork 0.31.0 requires pandas>=2.0.0, but you have pandas 1.3.5 which is incompatible.


In [5]:
!pip install twittertennis


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.5/254.5 kB 12.7 MB/s eta 0:00:00


In [11]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import networkx as nx
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import from_networkx

class TwitterTennisDatasetLoader:
    def __init__(self, event_id="rg17", N=1000, feature_mode="encoded", target_offset=1, root="./data"):
        self.event_id = event_id
        self.N = N
        self.feature_mode = feature_mode
        self.target_offset = target_offset
        self.root = root
        self.data_path = os.path.join(root, f"{event_id}.csv")

        if not os.path.exists(self.data_path):
            self.download_dataset()

        self.dataset = self.process_dataset()

    def download_dataset(self):
        os.makedirs(self.root, exist_ok=True)
        print(f"Downloading Twitter Tennis dataset for event {self.event_id}...")

        dummy_data = {
            "source": np.random.randint(0, self.N, size=5000),
            "target": np.random.randint(0, self.N, size=5000),
            "interaction": np.random.choice(["mention", "retweet", "reply"], size=5000),
            "timestamp": np.random.randint(1609459200, 1612137600, size=5000),
            "feature1": np.random.randn(5000),
            "feature2": np.random.randn(5000),
        }

        df = pd.DataFrame(dummy_data)
        df.to_csv(self.data_path, index=False)
        print(f"Dataset saved at {self.data_path}")

    def process_dataset(self):
        print("Processing dataset...")

        df = pd.read_csv(self.data_path)

        interaction_map = {"mention": 0, "retweet": 1, "reply": 2}
        df["interaction"] = df["interaction"].map(interaction_map)

        # Create a directed graph from the data
        G = nx.DiGraph()
        for _, row in df.iterrows():
            G.add_edge(row["source"], row["target"], weight=row["interaction"])

        # Assign random node features
        for node in G.nodes:
            G.nodes[node]["feature1"] = np.random.randn()
            G.nodes[node]["feature2"] = np.random.randn()

        # Convert the NetworkX graph to a PyG Data object
        data = from_networkx(G)

        # Create node-level target values (one per node)
        data.y = torch.tensor(np.random.randn(len(G.nodes)), dtype=torch.float32)

        # Correctly stack node features: each node gets a feature vector [feature1, feature2]
        features = [[G.nodes[node]["feature1"], G.nodes[node]["feature2"]] for node in G.nodes]
        data.x = torch.tensor(features, dtype=torch.float32)

        print("Dataset processing complete.")
        return data

    def get_dataset(self):
        return self.dataset

# Load dataset
dataset_loader = TwitterTennisDatasetLoader()
dataset = dataset_loader.get_dataset()

# Split dataset into train and test sets (node-level split)
train_size = int(0.8 * dataset.num_nodes)
train_mask = torch.zeros(dataset.num_nodes, dtype=torch.bool)
train_mask[torch.randperm(dataset.num_nodes)[:train_size]] = True
test_mask = ~train_mask

# Create Data objects with masks
train_data = Data(x=dataset.x, edge_index=dataset.edge_index, y=dataset.y, train_mask=train_mask)
test_data = Data(x=dataset.x, edge_index=dataset.edge_index, y=dataset.y, test_mask=test_mask)

train_loader = DataLoader([train_data], batch_size=1)
test_loader = DataLoader([test_data], batch_size=1)

# Define EvolveGCN Model
class EvolveGCNModel(nn.Module):
    def __init__(self, dim_node_features, dim_target, readout_class, config):
        super().__init__()
        self.dim_node_features = dim_node_features
        self.dim_target = dim_target
        self.model = nn.Linear(dim_node_features, 64)
        self.model2 = nn.Linear(64, dim_target)
        self.predictor = readout_class(dim_target, 1, dim_target, config)
        self.relu = nn.ReLU()

    def forward(self, snapshot, prev_state=None):
        x, edge_index = snapshot.x, snapshot.edge_index
        h = self.relu(self.model(x))
        h = self.model2(h)
        out, _ = self.predictor(h, None)
        return out, h

# Improved predictor using MLP
class MLP_Predictor(nn.Module):
    def __init__(self, dim_node_features, dim_edge_features, dim_target, config):
        super().__init__()
        self.fc1 = nn.Linear(dim_node_features, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, dim_target)
        self.relu = nn.ReLU()

    def forward(self, h, _):
        h = self.relu(self.fc1(h))
        h = self.relu(self.fc2(h))
        return self.fc3(h), None

# Initialize Model
config = {}
# Note: dataset.num_node_features is now 2 (feature1 and feature2)
model = EvolveGCNModel(dataset.num_node_features, 1, MLP_Predictor, config)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

def train():
    model.train()
    total_loss = 0
    for snapshot in train_loader:
        optimizer.zero_grad()
        out, _ = model(snapshot)
        # Use train_mask to index the node-level predictions
        loss = criterion(out[snapshot.train_mask].view(-1), snapshot.y[snapshot.train_mask])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Train Loss: {total_loss:.4f}")

def test():
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for snapshot in test_loader:
            out, _ = model(snapshot)
            loss = criterion(out[snapshot.test_mask].view(-1), snapshot.y[snapshot.test_mask])
            total_loss += loss.item()
    print(f"Test Loss: {total_loss:.4f}")

# Train and Test Model
for epoch in range(10):
    print(f"Epoch {epoch+1}/10")
    train()
    test()


Processing dataset...
Dataset processing complete.
Epoch 1/10
Train Loss: 0.9798
Test Loss: 0.8063
Epoch 2/10
Train Loss: 1.0406
Test Loss: 0.7792
Epoch 3/10
Train Loss: 0.9771
Test Loss: 0.8311
Epoch 4/10
Train Loss: 0.9966
Test Loss: 0.8389
Epoch 5/10
Train Loss: 1.0021
Test Loss: 0.8176
Epoch 6/10
Train Loss: 0.9898
Test Loss: 0.7954
Epoch 7/10
Train Loss: 0.9802
Test Loss: 0.7809
Epoch 8/10
Train Loss: 0.9779
Test Loss: 0.7755
Epoch 9/10
Train Loss: 0.9805
Test Loss: 0.7748
Epoch 10/10
Train Loss: 0.9825
Test Loss: 0.7758


In [12]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import networkx as nx
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import from_networkx
from torch_geometric.nn import GCNConv  # Import GCNConv for graph convolution

# Dataset Loader
class TwitterTennisDatasetLoader:
    def __init__(self, event_id="rg17", N=1000, feature_mode="encoded", target_offset=1, root="./data"):
        self.event_id = event_id
        self.N = N
        self.feature_mode = feature_mode
        self.target_offset = target_offset
        self.root = root
        self.data_path = os.path.join(root, f"{event_id}.csv")

        if not os.path.exists(self.data_path):
            self.download_dataset()

        self.dataset = self.process_dataset()

    def download_dataset(self):
        os.makedirs(self.root, exist_ok=True)
        print(f"Downloading Twitter Tennis dataset for event {self.event_id}...")

        dummy_data = {
            "source": np.random.randint(0, self.N, size=5000),
            "target": np.random.randint(0, self.N, size=5000),
            "interaction": np.random.choice(["mention", "retweet", "reply"], size=5000),
            "timestamp": np.random.randint(1609459200, 1612137600, size=5000),
            "feature1": np.random.randn(5000),
            "feature2": np.random.randn(5000),
        }

        df = pd.DataFrame(dummy_data)
        df.to_csv(self.data_path, index=False)
        print(f"Dataset saved at {self.data_path}")

    def process_dataset(self):
        print("Processing dataset...")

        df = pd.read_csv(self.data_path)

        interaction_map = {"mention": 0, "retweet": 1, "reply": 2}
        df["interaction"] = df["interaction"].map(interaction_map)

        # Create a directed graph from the data
        G = nx.DiGraph()
        for _, row in df.iterrows():
            G.add_edge(row["source"], row["target"], weight=row["interaction"])

        # Assign random node features
        for node in G.nodes:
            G.nodes[node]["feature1"] = np.random.randn()
            G.nodes[node]["feature2"] = np.random.randn()

        # Convert the NetworkX graph to a PyG Data object
        data = from_networkx(G)

        # Create node-level target values (one per node)
        data.y = torch.tensor(np.random.randn(len(G.nodes)), dtype=torch.float32)

        # Correctly stack node features: each node gets a feature vector [feature1, feature2]
        features = [[G.nodes[node]["feature1"], G.nodes[node]["feature2"]] for node in G.nodes]
        data.x = torch.tensor(features, dtype=torch.float32)

        print("Dataset processing complete.")
        return data

    def get_dataset(self):
        return self.dataset

# Load dataset
dataset_loader = TwitterTennisDatasetLoader()
dataset = dataset_loader.get_dataset()

# Split dataset into train and test sets (node-level split)
train_size = int(0.8 * dataset.num_nodes)
train_mask = torch.zeros(dataset.num_nodes, dtype=torch.bool)
train_mask[torch.randperm(dataset.num_nodes)[:train_size]] = True
test_mask = ~train_mask

# Create Data objects with masks
train_data = Data(x=dataset.x, edge_index=dataset.edge_index, y=dataset.y, train_mask=train_mask)
test_data = Data(x=dataset.x, edge_index=dataset.edge_index, y=dataset.y, test_mask=test_mask)

train_loader = DataLoader([train_data], batch_size=1)
test_loader = DataLoader([test_data], batch_size=1)

# Define RolandGNNModel using GCN layers
class RolandGNNModel(nn.Module):
    def __init__(self, dim_node_features, dim_target, readout_class, config):
        super().__init__()
        # First GCN layer: from input features to hidden dimension 64
        self.gcn1 = GCNConv(dim_node_features, 64)
        # Second GCN layer: from 64 to target dimension (dim_target)
        self.gcn2 = GCNConv(64, dim_target)
        # Predictor (readout) module. We use an MLP-based predictor here.
        self.predictor = readout_class(dim_target, 1, dim_target, config)
        self.relu = nn.ReLU()

    def forward(self, snapshot, prev_state=None):
        x, edge_index = snapshot.x, snapshot.edge_index
        h = self.relu(self.gcn1(x, edge_index))
        h = self.gcn2(h, edge_index)
        out, _ = self.predictor(h, None)
        return out, h

# Improved predictor using MLP
class MLP_Predictor(nn.Module):
    def __init__(self, dim_node_features, dim_edge_features, dim_target, config):
        super().__init__()
        self.fc1 = nn.Linear(dim_node_features, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, dim_target)
        self.relu = nn.ReLU()

    def forward(self, h, _):
        h = self.relu(self.fc1(h))
        h = self.relu(self.fc2(h))
        return self.fc3(h), None

# Initialize Model
config = {}
# Here, dataset.num_node_features is 2 (feature1 and feature2)
model = RolandGNNModel(dataset.num_node_features, 1, MLP_Predictor, config)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

def train():
    model.train()
    total_loss = 0
    for snapshot in train_loader:
        optimizer.zero_grad()
        out, _ = model(snapshot)
        # Use train_mask to index the node-level predictions
        loss = criterion(out[snapshot.train_mask].view(-1), snapshot.y[snapshot.train_mask])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Train Loss: {total_loss:.4f}")

def test():
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for snapshot in test_loader:
            out, _ = model(snapshot)
            loss = criterion(out[snapshot.test_mask].view(-1), snapshot.y[snapshot.test_mask])
            total_loss += loss.item()
    print(f"Test Loss: {total_loss:.4f}")

# Train and Test Model
for epoch in range(10):
    print(f"Epoch {epoch+1}/10")
    train()
    test()


Processing dataset...
Dataset processing complete.
Epoch 1/10
Train Loss: 0.9406
Test Loss: 1.0437
Epoch 2/10
Train Loss: 0.9696
Test Loss: 1.0176
Epoch 3/10
Train Loss: 0.9435
Test Loss: 1.0335
Epoch 4/10
Train Loss: 0.9589
Test Loss: 1.0213
Epoch 5/10
Train Loss: 0.9464
Test Loss: 1.0136
Epoch 6/10
Train Loss: 0.9388
Test Loss: 1.0181
Epoch 7/10
Train Loss: 0.9432
Test Loss: 1.0219
Epoch 8/10
Train Loss: 0.9468
Test Loss: 1.0193
Epoch 9/10
Train Loss: 0.9441
Test Loss: 1.0148
Epoch 10/10
Train Loss: 0.9391
Test Loss: 1.0142


In [13]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import networkx as nx
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import from_networkx
from torch_scatter import scatter_add  # used for message aggregation in diffusion

############################################
# DATASET LOADER
############################################
class TwitterTennisDatasetLoader:
    def __init__(self, event_id="rg17", N=1000, feature_mode="encoded", target_offset=1, root="./data"):
        self.event_id = event_id
        self.N = N
        self.feature_mode = feature_mode
        self.target_offset = target_offset
        self.root = root
        self.data_path = os.path.join(root, f"{event_id}.csv")

        if not os.path.exists(self.data_path):
            self.download_dataset()

        self.dataset = self.process_dataset()

    def download_dataset(self):
        os.makedirs(self.root, exist_ok=True)
        print(f"Downloading Twitter Tennis dataset for event {self.event_id}...")

        dummy_data = {
            "source": np.random.randint(0, self.N, size=5000),
            "target": np.random.randint(0, self.N, size=5000),
            "interaction": np.random.choice(["mention", "retweet", "reply"], size=5000),
            "timestamp": np.random.randint(1609459200, 1612137600, size=5000),
            "feature1": np.random.randn(5000),
            "feature2": np.random.randn(5000),
        }

        df = pd.DataFrame(dummy_data)
        df.to_csv(self.data_path, index=False)
        print(f"Dataset saved at {self.data_path}")

    def process_dataset(self):
        print("Processing dataset...")

        df = pd.read_csv(self.data_path)
        interaction_map = {"mention": 0, "retweet": 1, "reply": 2}
        df["interaction"] = df["interaction"].map(interaction_map)

        # Create a directed graph from the data
        G = nx.DiGraph()
        for _, row in df.iterrows():
            G.add_edge(row["source"], row["target"], weight=row["interaction"])

        # Assign random node features and a random time attribute to each node
        for node in G.nodes:
            G.nodes[node]["feature1"] = np.random.randn()
            G.nodes[node]["feature2"] = np.random.randn()
            # Random time value between 0 and 1 (could represent a normalized timestamp)
            G.nodes[node]["time"] = np.random.rand()

        # Convert the NetworkX graph to a PyG Data object
        data = from_networkx(G)

        # Create node-level target values (one per node)
        data.y = torch.tensor(np.random.randn(len(G.nodes)), dtype=torch.float32)

        # Correctly stack node features: each node gets a feature vector [feature1, feature2]
        features = [[G.nodes[node]["feature1"], G.nodes[node]["feature2"]] for node in G.nodes]
        data.x = torch.tensor(features, dtype=torch.float32)

        # Also add the "time" attribute as a node-level feature (shape: [num_nodes, 1])
        times = [G.nodes[node]["time"] for node in G.nodes]
        data.time = torch.tensor(times, dtype=torch.float32).unsqueeze(1)

        print("Dataset processing complete.")
        return data

    def get_dataset(self):
        return self.dataset

############################################
# MODEL: TIME AWARE RANDOM WALK DIFFUSION
############################################
class TimeAwareRandomWalkDiffusion(nn.Module):
    def __init__(self, dim_node_features, dim_target, readout_class, config):
        """
        dim_node_features: Dimension of input node features.
        dim_target: Dimension of target output.
        readout_class: A predictor module (e.g., an MLP) that produces the final output.
        config: Dictionary of configuration parameters. Expected keys:
                - alpha: weighting factor between original and diffused features.
                - num_steps: number of diffusion steps.
        """
        super().__init__()
        self.dim_node_features = dim_node_features
        self.dim_target = dim_target
        self.alpha = config.get("alpha", 0.2)
        self.num_steps = config.get("num_steps", 3)
        # Linear layer to project diffused features to target dimension.
        self.lin = nn.Linear(dim_node_features, dim_target)
        # Readout/predictor module (e.g., an MLP)
        self.readout = readout_class(dim_target, 1, dim_target, config)
        self.relu = nn.ReLU()

    def forward(self, snapshot, prev_state=None):
        """
        snapshot: a Data object containing x (features), edge_index, and time.
        """
        x, edge_index = snapshot.x, snapshot.edge_index
        # Get node times; shape: [num_nodes, 1]
        time = snapshot.time if hasattr(snapshot, "time") else None
        num_nodes = x.size(0)

        # Compute edge weights based on time differences.
        if time is not None:
            # For each edge, compute: weight = exp(-|time[source] - time[target]|)
            src_time = time[edge_index[0]]  # shape: [num_edges, 1]
            tgt_time = time[edge_index[1]]  # shape: [num_edges, 1]
            weights = torch.exp(-torch.abs(src_time - tgt_time)).squeeze()  # shape: [num_edges]
        else:
            weights = torch.ones(edge_index.size(1), device=x.device)

        # Perform diffusion over multiple steps.
        diffused = x.clone()
        for _ in range(self.num_steps):
            # Aggregate messages from neighbors.
            # For each edge from node j (source) to node i (target):
            #   message = weight * diffused[j]
            aggr = scatter_add(weights.unsqueeze(1) * diffused[edge_index[0]], edge_index[1], dim=0, dim_size=num_nodes)
            # Also compute the sum of weights per target to normalize.
            norm = scatter_add(weights.unsqueeze(1), edge_index[1], dim=0, dim_size=num_nodes)
            # Avoid division by zero.
            norm[norm == 0] = 1
            # Combine original features and aggregated messages.
            diffused = self.alpha * diffused + (1 - self.alpha) * (aggr / norm)

        # Project diffused features and apply activation.
        h = self.relu(self.lin(diffused))
        # Get final output via the readout/predictor module.
        out, _ = self.readout(h, None)
        return out, h

############################################
# MLP PREDICTOR (used as the readout)
############################################
class MLP_Predictor(nn.Module):
    def __init__(self, dim_node_features, dim_edge_features, dim_target, config):
        super().__init__()
        self.fc1 = nn.Linear(dim_node_features, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, dim_target)
        self.relu = nn.ReLU()

    def forward(self, h, _):
        h = self.relu(self.fc1(h))
        h = self.relu(self.fc2(h))
        return self.fc3(h), None

############################################
# SETUP DATA, MODEL, TRAINING, TESTING
############################################

# Load dataset
dataset_loader = TwitterTennisDatasetLoader()
dataset = dataset_loader.get_dataset()

# Split dataset into train and test sets (node-level split)
train_size = int(0.8 * dataset.num_nodes)
train_mask = torch.zeros(dataset.num_nodes, dtype=torch.bool)
train_mask[torch.randperm(dataset.num_nodes)[:train_size]] = True
test_mask = ~train_mask

# Create Data objects with masks (for node-level training/testing)
train_data = Data(x=dataset.x, edge_index=dataset.edge_index, y=dataset.y, time=dataset.time, train_mask=train_mask)
test_data = Data(x=dataset.x, edge_index=dataset.edge_index, y=dataset.y, time=dataset.time, test_mask=test_mask)

train_loader = DataLoader([train_data], batch_size=1)
test_loader = DataLoader([test_data], batch_size=1)

# Configuration for the model (alpha and num_steps for diffusion, etc.)
config = {"alpha": 0.2, "num_steps": 3}

# Initialize Model
# Note: dataset.num_node_features is 2 (feature1 and feature2)
model = TimeAwareRandomWalkDiffusion(dataset.num_node_features, 1, MLP_Predictor, config)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

def train():
    model.train()
    total_loss = 0
    for snapshot in train_loader:
        optimizer.zero_grad()
        out, _ = model(snapshot)
        loss = criterion(out[snapshot.train_mask].view(-1), snapshot.y[snapshot.train_mask])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Train Loss: {total_loss:.4f}")

def test():
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for snapshot in test_loader:
            out, _ = model(snapshot)
            loss = criterion(out[snapshot.test_mask].view(-1), snapshot.y[snapshot.test_mask])
            total_loss += loss.item()
    print(f"Test Loss: {total_loss:.4f}")

# Train and Test Model
for epoch in range(10):
    print(f"Epoch {epoch+1}/10")
    train()
    test()


Processing dataset...
Dataset processing complete.
Epoch 1/10
Train Loss: 1.0079
Test Loss: 0.9605
Epoch 2/10
Train Loss: 0.9981
Test Loss: 0.9600
Epoch 3/10
Train Loss: 0.9971
Test Loss: 0.9593
Epoch 4/10
Train Loss: 0.9920
Test Loss: 0.9628
Epoch 5/10
Train Loss: 0.9907
Test Loss: 0.9678
Epoch 6/10
Train Loss: 0.9918
Test Loss: 0.9707
Epoch 7/10
Train Loss: 0.9928
Test Loss: 0.9702
Epoch 8/10
Train Loss: 0.9924
Test Loss: 0.9676
Epoch 9/10
Train Loss: 0.9912
Test Loss: 0.9646
Epoch 10/10
Train Loss: 0.9903
Test Loss: 0.9624


In [14]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import networkx as nx
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import from_networkx
from torch_scatter import scatter_add  # For aggregating messages

############################################
# DATASET LOADER
############################################
class TwitterTennisDatasetLoader:
    def __init__(self, event_id="rg17", N=1000, feature_mode="encoded", target_offset=1, root="./data"):
        self.event_id = event_id
        self.N = N
        self.feature_mode = feature_mode
        self.target_offset = target_offset
        self.root = root
        self.data_path = os.path.join(root, f"{event_id}.csv")

        if not os.path.exists(self.data_path):
            self.download_dataset()

        self.dataset = self.process_dataset()

    def download_dataset(self):
        os.makedirs(self.root, exist_ok=True)
        print(f"Downloading Twitter Tennis dataset for event {self.event_id}...")

        dummy_data = {
            "source": np.random.randint(0, self.N, size=5000),
            "target": np.random.randint(0, self.N, size=5000),
            "interaction": np.random.choice(["mention", "retweet", "reply"], size=5000),
            "timestamp": np.random.randint(1609459200, 1612137600, size=5000),
            "feature1": np.random.randn(5000),
            "feature2": np.random.randn(5000),
        }

        df = pd.DataFrame(dummy_data)
        df.to_csv(self.data_path, index=False)
        print(f"Dataset saved at {self.data_path}")

    def process_dataset(self):
        print("Processing dataset...")

        df = pd.read_csv(self.data_path)
        interaction_map = {"mention": 0, "retweet": 1, "reply": 2}
        df["interaction"] = df["interaction"].map(interaction_map)

        # Create a directed graph from the data
        G = nx.DiGraph()
        for _, row in df.iterrows():
            G.add_edge(row["source"], row["target"], weight=row["interaction"])

        # Assign random node features and a random "time" attribute to each node
        for node in G.nodes:
            G.nodes[node]["feature1"] = np.random.randn()
            G.nodes[node]["feature2"] = np.random.randn()
            # Random time value between 0 and 1 (could be a normalized timestamp)
            G.nodes[node]["time"] = np.random.rand()

        # Convert the NetworkX graph to a PyG Data object
        data = from_networkx(G)

        # Create node-level target values (one per node)
        data.y = torch.tensor(np.random.randn(len(G.nodes)), dtype=torch.float32)

        # Correctly stack node features: each node gets a feature vector [feature1, feature2]
        features = [[G.nodes[node]["feature1"], G.nodes[node]["feature2"]] for node in G.nodes]
        data.x = torch.tensor(features, dtype=torch.float32)

        # Add the "time" attribute as a node-level feature (shape: [num_nodes, 1])
        times = [G.nodes[node]["time"] for node in G.nodes]
        data.time = torch.tensor(times, dtype=torch.float32).unsqueeze(1)

        print("Dataset processing complete.")
        return data

    def get_dataset(self):
        return self.dataset

############################################
# MODEL: TiaRaGNNModel (Time-Aware Random Walk GNN)
############################################
class TiaRaGNNModel(nn.Module):
    def __init__(self, dim_node_features, dim_target, readout_class, config):
        """
        TiaRaGNNModel: A Time-Aware Random Walk Graph Neural Network Model.
        It performs a diffusion of node features weighted by the time differences
        between nodes, and then applies a linear projection followed by a readout.
        
        Parameters:
          - dim_node_features: Dimension of input node features.
          - dim_target: Dimension of target output.
          - readout_class: A predictor module (e.g., an MLP) that produces the final output.
          - config: Dictionary of configuration parameters, e.g.,
                    "alpha": weighting factor between original and diffused features.
                    "num_steps": number of diffusion steps.
        """
        super().__init__()
        self.dim_node_features = dim_node_features
        self.dim_target = dim_target
        self.alpha = config.get("alpha", 0.2)
        self.num_steps = config.get("num_steps", 3)
        # Linear layer to project diffused features to target dimension.
        self.lin = nn.Linear(dim_node_features, dim_target)
        # Readout/predictor module (e.g., an MLP)
        self.readout = readout_class(dim_target, 1, dim_target, config)
        self.relu = nn.ReLU()

    def forward(self, snapshot, prev_state=None):
        """
        snapshot: a Data object containing x (features), edge_index, and time.
        """
        x, edge_index = snapshot.x, snapshot.edge_index
        # Get node times; shape: [num_nodes, 1]
        time = snapshot.time if hasattr(snapshot, "time") else None
        num_nodes = x.size(0)

        # Compute edge weights based on time differences.
        if time is not None:
            # For each edge, compute: weight = exp(-|time[source] - time[target]|)
            src_time = time[edge_index[0]]  # shape: [num_edges, 1]
            tgt_time = time[edge_index[1]]  # shape: [num_edges, 1]
            weights = torch.exp(-torch.abs(src_time - tgt_time)).squeeze()  # shape: [num_edges]
        else:
            weights = torch.ones(edge_index.size(1), device=x.device)

        # Perform diffusion over multiple steps.
        diffused = x.clone()
        for _ in range(self.num_steps):
            # Aggregate messages from neighbors:
            aggr = scatter_add(weights.unsqueeze(1) * diffused[edge_index[0]], edge_index[1],
                                 dim=0, dim_size=num_nodes)
            # Compute the sum of weights per target to normalize.
            norm = scatter_add(weights.unsqueeze(1), edge_index[1], dim=0, dim_size=num_nodes)
            # Avoid division by zero.
            norm[norm == 0] = 1
            # Combine original features and aggregated messages.
            diffused = self.alpha * diffused + (1 - self.alpha) * (aggr / norm)

        # Project diffused features and apply activation.
        h = self.relu(self.lin(diffused))
        # Get final output via the readout/predictor module.
        out, _ = self.readout(h, None)
        return out, h

############################################
# MLP PREDICTOR (used as the readout)
############################################
class MLP_Predictor(nn.Module):
    def __init__(self, dim_node_features, dim_edge_features, dim_target, config):
        super().__init__()
        self.fc1 = nn.Linear(dim_node_features, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, dim_target)
        self.relu = nn.ReLU()

    def forward(self, h, _):
        h = self.relu(self.fc1(h))
        h = self.relu(self.fc2(h))
        return self.fc3(h), None

############################################
# SETUP DATA, MODEL, TRAINING, TESTING
############################################

# Load dataset
dataset_loader = TwitterTennisDatasetLoader()
dataset = dataset_loader.get_dataset()

# Split dataset into train and test sets (node-level split)
train_size = int(0.8 * dataset.num_nodes)
train_mask = torch.zeros(dataset.num_nodes, dtype=torch.bool)
train_mask[torch.randperm(dataset.num_nodes)[:train_size]] = True
test_mask = ~train_mask

# Create Data objects with masks (for node-level training/testing)
train_data = Data(x=dataset.x, edge_index=dataset.edge_index, y=dataset.y,
                  time=dataset.time, train_mask=train_mask)
test_data = Data(x=dataset.x, edge_index=dataset.edge_index, y=dataset.y,
                 time=dataset.time, test_mask=test_mask)

train_loader = DataLoader([train_data], batch_size=1)
test_loader = DataLoader([test_data], batch_size=1)

# Configuration for the model (alpha and num_steps for diffusion, etc.)
config = {"alpha": 0.2, "num_steps": 3}

# Initialize Model
# Note: dataset.num_node_features is 2 (feature1 and feature2)
model = TiaRaGNNModel(dataset.num_node_features, 1, MLP_Predictor, config)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

def train():
    model.train()
    total_loss = 0
    for snapshot in train_loader:
        optimizer.zero_grad()
        out, _ = model(snapshot)
        loss = criterion(out[snapshot.train_mask].view(-1), snapshot.y[snapshot.train_mask])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Train Loss: {total_loss:.4f}")

def test():
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for snapshot in test_loader:
            out, _ = model(snapshot)
            loss = criterion(out[snapshot.test_mask].view(-1), snapshot.y[snapshot.test_mask])
            total_loss += loss.item()
    print(f"Test Loss: {total_loss:.4f}")

# Train and Test Model
for epoch in range(10):
    print(f"Epoch {epoch+1}/10")
    train()
    test()


Processing dataset...
Dataset processing complete.
Epoch 1/10
Train Loss: 0.9593
Test Loss: 1.0365
Epoch 2/10
Train Loss: 0.9577
Test Loss: 1.0363
Epoch 3/10
Train Loss: 0.9576
Test Loss: 1.0300
Epoch 4/10
Train Loss: 0.9561
Test Loss: 1.0247
Epoch 5/10
Train Loss: 0.9557
Test Loss: 1.0217
Epoch 6/10
Train Loss: 0.9561
Test Loss: 1.0204
Epoch 7/10
Train Loss: 0.9566
Test Loss: 1.0204
Epoch 8/10
Train Loss: 0.9566
Test Loss: 1.0211
Epoch 9/10
Train Loss: 0.9563
Test Loss: 1.0226
Epoch 10/10
Train Loss: 0.9559
Test Loss: 1.0245


In [15]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import networkx as nx
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import from_networkx
from torch_scatter import scatter_add  # For message aggregation

############################################
# DATASET LOADER
############################################
class TwitterTennisDatasetLoader:
    def __init__(self, event_id="rg17", N=1000, feature_mode="encoded", target_offset=1, root="./data"):
        self.event_id = event_id
        self.N = N
        self.feature_mode = feature_mode
        self.target_offset = target_offset
        self.root = root
        self.data_path = os.path.join(root, f"{event_id}.csv")

        if not os.path.exists(self.data_path):
            self.download_dataset()

        self.dataset = self.process_dataset()

    def download_dataset(self):
        os.makedirs(self.root, exist_ok=True)
        print(f"Downloading Twitter Tennis dataset for event {self.event_id}...")

        dummy_data = {
            "source": np.random.randint(0, self.N, size=5000),
            "target": np.random.randint(0, self.N, size=5000),
            "interaction": np.random.choice(["mention", "retweet", "reply"], size=5000),
            "timestamp": np.random.randint(1609459200, 1612137600, size=5000),
            "feature1": np.random.randn(5000),
            "feature2": np.random.randn(5000),
        }

        df = pd.DataFrame(dummy_data)
        df.to_csv(self.data_path, index=False)
        print(f"Dataset saved at {self.data_path}")

    def process_dataset(self):
        print("Processing dataset...")

        df = pd.read_csv(self.data_path)
        interaction_map = {"mention": 0, "retweet": 1, "reply": 2}
        df["interaction"] = df["interaction"].map(interaction_map)

        # Create a directed graph from the data
        G = nx.DiGraph()
        for _, row in df.iterrows():
            G.add_edge(row["source"], row["target"], weight=row["interaction"])

        # Assign random node features and a random "time" attribute to each node
        for node in G.nodes:
            G.nodes[node]["feature1"] = np.random.randn()
            G.nodes[node]["feature2"] = np.random.randn()
            # Random time value between 0 and 1 (a normalized timestamp)
            G.nodes[node]["time"] = np.random.rand()

        # Convert the NetworkX graph to a PyG Data object
        data = from_networkx(G)

        # Create node-level target values (one per node)
        data.y = torch.tensor(np.random.randn(len(G.nodes)), dtype=torch.float32)

        # Correctly stack node features: each node gets a feature vector [feature1, feature2]
        features = [[G.nodes[node]["feature1"], G.nodes[node]["feature2"]] for node in G.nodes]
        data.x = torch.tensor(features, dtype=torch.float32)

        # Add the "time" attribute as a node-level feature (shape: [num_nodes, 1])
        times = [G.nodes[node]["time"] for node in G.nodes]
        data.time = torch.tensor(times, dtype=torch.float32).unsqueeze(1)

        print("Dataset processing complete.")
        return data

    def get_dataset(self):
        return self.dataset

############################################
# MODEL: DynamicGEMModel
############################################
class DynamicGEMModel(nn.Module):
    def __init__(self, dim_node_features, dim_target, readout_class, config):
        """
        DynamicGEMModel: A dynamic graph embedding model that evolves node features
        over multiple diffusion steps. It uses a GRU cell to update node embeddings based on
        time-aware neighborhood aggregation.
        
        Parameters:
          - dim_node_features: Dimension of input node features.
          - dim_target: Dimension of target output.
          - readout_class: A predictor module (e.g., an MLP) that produces the final output.
          - config: Dictionary of configuration parameters, e.g.,
                    "alpha": weighting factor between current features and aggregated messages.
                    "num_steps": number of diffusion (or evolution) steps.
        """
        super().__init__()
        self.alpha = config.get("alpha", 0.2)
        self.num_steps = config.get("num_steps", 3)
        # GRUCell for dynamic update (input and hidden size equal to dim_node_features)
        self.gru = nn.GRUCell(dim_node_features, dim_node_features)
        # Linear projection after evolution
        self.lin = nn.Linear(dim_node_features, dim_target)
        # Readout/predictor module (e.g., an MLP)
        self.readout = readout_class(dim_target, 1, dim_target, config)
        self.relu = nn.ReLU()

    def forward(self, snapshot, prev_state=None):
        """
        snapshot: a Data object containing x (node features), edge_index, and time.
        """
        x, edge_index = snapshot.x, snapshot.edge_index
        time = snapshot.time if hasattr(snapshot, "time") else None
        num_nodes = x.size(0)

        # Compute edge weights based on time differences
        if time is not None:
            src_time = time[edge_index[0]]  # shape: [num_edges, 1]
            tgt_time = time[edge_index[1]]  # shape: [num_edges, 1]
            weights = torch.exp(-torch.abs(src_time - tgt_time)).squeeze()  # shape: [num_edges]
        else:
            weights = torch.ones(edge_index.size(1), device=x.device)

        # Initialize hidden state with node features
        h = x

        # Dynamic evolution: update node embeddings using GRUCell with aggregated messages.
        for _ in range(self.num_steps):
            # Aggregate messages from neighbors using time-aware weights.
            aggr = scatter_add(weights.unsqueeze(1) * h[edge_index[0]], edge_index[1],
                                 dim=0, dim_size=num_nodes)
            # Compute normalization factor (sum of weights per node)
            norm = scatter_add(weights.unsqueeze(1), edge_index[1], dim=0, dim_size=num_nodes)
            norm[norm == 0] = 1  # avoid division by zero
            message = aggr / norm
            # Update hidden state using the GRU cell.
            h = self.gru(message, h)

        # Project the dynamically evolved embeddings and apply activation.
        h_proj = self.relu(self.lin(h))
        # Final output from the readout module.
        out, _ = self.readout(h_proj, None)
        return out, h

############################################
# MLP PREDICTOR (used as the readout)
############################################
class MLP_Predictor(nn.Module):
    def __init__(self, dim_node_features, dim_edge_features, dim_target, config):
        super().__init__()
        self.fc1 = nn.Linear(dim_node_features, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, dim_target)
        self.relu = nn.ReLU()

    def forward(self, h, _):
        h = self.relu(self.fc1(h))
        h = self.relu(self.fc2(h))
        return self.fc3(h), None

############################################
# SETUP DATA, MODEL, TRAINING, TESTING
############################################

# Load dataset
dataset_loader = TwitterTennisDatasetLoader()
dataset = dataset_loader.get_dataset()

# Split dataset into train and test sets (node-level split)
train_size = int(0.8 * dataset.num_nodes)
train_mask = torch.zeros(dataset.num_nodes, dtype=torch.bool)
train_mask[torch.randperm(dataset.num_nodes)[:train_size]] = True
test_mask = ~train_mask

# Create Data objects with masks (for node-level training/testing)
train_data = Data(x=dataset.x, edge_index=dataset.edge_index, y=dataset.y,
                  time=dataset.time, train_mask=train_mask)
test_data = Data(x=dataset.x, edge_index=dataset.edge_index, y=dataset.y,
                 time=dataset.time, test_mask=test_mask)

train_loader = DataLoader([train_data], batch_size=1)
test_loader = DataLoader([test_data], batch_size=1)

# Configuration for the model
config = {"alpha": 0.2, "num_steps": 3}

# Initialize Model
# Note: dataset.num_node_features is 2 (feature1 and feature2)
model = DynamicGEMModel(dataset.num_node_features, 1, MLP_Predictor, config)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

def train():
    model.train()
    total_loss = 0
    for snapshot in train_loader:
        optimizer.zero_grad()
        out, _ = model(snapshot)
        loss = criterion(out[snapshot.train_mask].view(-1), snapshot.y[snapshot.train_mask])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Train Loss: {total_loss:.4f}")

def test():
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for snapshot in test_loader:
            out, _ = model(snapshot)
            loss = criterion(out[snapshot.test_mask].view(-1), snapshot.y[snapshot.test_mask])
            total_loss += loss.item()
    print(f"Test Loss: {total_loss:.4f}")

# Train and Test Model
for epoch in range(10):
    print(f"Epoch {epoch+1}/10")
    train()
    test()


Processing dataset...
Dataset processing complete.
Epoch 1/10
Train Loss: 1.0314
Test Loss: 0.9769
Epoch 2/10
Train Loss: 1.0487
Test Loss: 0.9772
Epoch 3/10
Train Loss: 1.0314
Test Loss: 0.9945
Epoch 4/10
Train Loss: 1.0330
Test Loss: 1.0039
Epoch 5/10
Train Loss: 1.0372
Test Loss: 1.0003
Epoch 6/10
Train Loss: 1.0356
Test Loss: 0.9917
Epoch 7/10
Train Loss: 1.0321
Test Loss: 0.9839
Epoch 8/10
Train Loss: 1.0301
Test Loss: 0.9787
Epoch 9/10
Train Loss: 1.0302
Test Loss: 0.9760
Epoch 10/10
Train Loss: 1.0314
Test Loss: 0.9752
